<h1><center>Stroke Prediction</center></h1>
<center>September 2024</center>
<center>Celine Ng</center>

# Table of Contents

1. Project Introduction   
    1. Notebook Preparation
    1. Motivation and Objectives
    1. Dataset
1. Data Cleaning
    1. Duplicate rows
    1. Datatypes
    1. Unique values
    1. Missing values
1. EDA
    1. Distribution
    1. Distribution according to target label
    1. Missing Values
    1. Statistic Inference
        1. Target Population
        2. Transform 'AnnualIncome'
       3. Hypothesis Testing
1. Data Formatting
1. Preprocessing
    1. Transformations
    1. Data Splitting
1. Models
    1. Basic model
    1. Baseline model
    1. Hyperparameter Tuning
1. Conclusion
1. Improvements

# 1. Project Introduction

## 1.1. Notebook Preparation

In [ ]:
%%capture
%pip install -r requirements.txt

In [ ]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from utils.eda import *
from utils.model import *

from sklearn.compose import make_column_selector as selector
from sklearn.model_selection import (train_test_split, StratifiedKFold, 
                                     cross_validate, GridSearchCV)

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (OneHotEncoder, FunctionTransformer,
                                   StandardScaler)
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import (accuracy_score, roc_auc_score, f1_score, 
                             precision_score, recall_score, make_scorer)

## 1.2. Motivations and Objectives

This project's objectives are: 
<br><br>
1. Practice performing EDA.
2. Practice applying statistical inference procedures.
3. Practice using various types of machine learning models.
4. Practice building ensembles of machine learning models.
5. Practice deploying machine learning models.

## 1.3. Dataset

Objective: Brief overview of our dataset, including the features and label

The dataset was downloaded from Kaggle, [Stroke_Prediction_Data](https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset)
,<br>
on 10 September 2024. It will be used to predict whether a patient is likely
 <br> to get a stroke. <br>
The data contains 11 clinical features, like gender, age, smoking status, <br>
etc, that help describe each patient.

Features: <br>
1) id: unique identifier
2) gender: "Male", "Female" or "Other"
3) age: age of the patient
4) hypertension: 0 if the patient doesn't have hypertension, 1 if the patient has hypertension
5) heart_disease: 0 if the patient doesn't have any heart diseases, 1 if the patient has a heart disease
6) ever_married: "No" or "Yes"
7) work_type: "children", "Govt_jov", "Never_worked", "Private" or "Self-employed"
8) Residence_type: "Rural" or "Urban"
9) avg_glucose_level: average glucose level in blood
10) bmi: body mass index
11) smoking_status: "formerly smoked", "never smoked", "smokes" or "Unknown"*
<br>

Label: <br>
stroke: 1 if the patient had a stroke or 0 if not <br>

In [ ]:
stroke_data = pd.read_csv('data/healthcare-dataset-stroke-data.csv')
display(stroke_data.head())
shape = stroke_data.shape
print(f'Number of rows: {shape[0]}\nNumber of columns: {shape[1]}')

# 2.  Data cleaning
Objective:
1. Closer look at the values that consist of our data
2. Look out for duplicates, and missing and/or unusual values

## 2.1. Duplicate rows

In [ ]:
print(stroke_data.id.duplicated().any())

After confirming there is no duplicated id number/cases, since id number 
should not be relevant information to base our prediction on, we can remove it.

In [ ]:
stroke_data = stroke_data.drop(columns=['id'])

## 2.2. Datatypes

In [ ]:
datatype_data = stroke_data.dtypes
datatype_data

Certain features have unexpected datatypes, like age, hypertension, and 
heart_disease. Looking into the values itself will help clarify data types.

## 2.3. Unique values

In [ ]:
stroke_data.nunique()

Some features have unexpected amount of unique values. Looking into the 
values itself will help.<br>

In [ ]:
for column in stroke_data.columns:
    df = stroke_data[column].value_counts()
    display(df)

Looking more into work_type registered as 'children', as the difference 
between 'children' and 'Never_worked' is not clear.

In [ ]:
stroke_data[stroke_data.work_type=='children'].age.describe()

All labeled as children are <= 16 years old.

In [ ]:
stroke_data[(stroke_data.age<=16)]['work_type'].value_counts()

It is clear that not all under 16 year olds are labeled as 'children'. Some 
are 'Never_worked', and some are working. It is unclear why only part is 
labeled as 'children'. <br>However, it since this
 feature is to state their work status and 'children' in this context would 
 mean not working, we can consolidate 'children' into 'Never_worked'. <br>
 This would ensure consistency across all age groups, without losing the age
  information, as it is in the 'age' column.

In [ ]:
stroke_data.loc[stroke_data['work_type'] == 'children', 'work_type'] = \
    'Never_worked'

## 2.4. Missing values

In [ ]:
missing_values(df=stroke_data)

To properly impute missing values, it is necessary to first understand the 
sampling population and if the missing values belong to a specific subset of
 sampling population.

In [ ]:
HTML('''
<div class="alert alert-block alert-info">
    <b>Data Cleaning Insights:</b><br>
    1. There are no duplicated rows, and each row is identified by a unique ID.<br>
    2. The feature 'id' is irrelevant to our stroke prediction, so it was removed.<br>
    3. Consolidated 'children' work type to 'Never_worked'. <br>
    4. All missing values come from the 'bmi' feature, and further analysis 
    is needed to determine proper imputation methods.
</div>
''')

# 3. EDA

Objectives:
1. Data distribution
2. Check and handle outliers, missing values
3. Comparison between data with and without stroke
4. Understand how our data was collected and possible bias
5. Understand how do columns related with each other - correlation
6. 

## 3.1. Distribution

Identify features, label, and different feature types.

In [ ]:
target_name = 'stroke'
data, target = (stroke_data.drop(columns=[target_name]), 
                stroke_data[target_name])

In [ ]:
categorical_columns_selector = selector(dtype_exclude='float64')
numerical_columns_selector = selector(dtype_include='float64')

categorical_columns = categorical_columns_selector(data)
numerical_columns = numerical_columns_selector(data)

**Numerical Features**

In [ ]:
data[numerical_columns].describe()

Plotting boxplot over violin plot for numerical features to have a visual 
understanding of the overall distribution, range, and outliers.

In [ ]:
plt.figure(figsize=(5,4))
violin_boxplot(data=data, columns=numerical_columns, title='Numerical Data '
                                                           'Violin Box '
                                                           'Plot')
plt.show()

**Categorical Features**

In [ ]:
categorical_columns

Represent categorical features in barplots, visualize unique values for each
 feature and their distribution in percentage.

In [ ]:
percentage_subplots(data=data, columns=categorical_columns, title='Categorical '
                                                             'data '
                                                             'Distribution',
               nrows=3, ncols=3)

**Target Variable**

Target variable and its distribution in percentage.

In [ ]:
percentage_subplots(data=stroke_data, columns=['stroke'], title='Target Variable')

In [ ]:
HTML('''
<div class="alert alert-block alert-info">
    <b>Distribution Insights:</b><br>
    <i>numerical features...</i><br>
    1. Numerical features show very different ranges of values, signalling an 
    importance in scaling in future feature engineering.<br>
    2. Average Glucose Level has the most outliers, which can be justified 
    by the bimodal distribution. This could contain relevant information to 
    our target prediction.
    <br>
    3. 'age' includes children less than 1 year old, justifies why the 
    datatype is float.<br><br>
    
    <i>for categorical features...</i><br>
    4. Inconsistent labeling among categorical features, preprocessing 
    required before applying learning model.<br>
    5. Outlier responses in 'gender'.<br><br>
    
    <i>and the target variable...</i><br>
    6. Imbalanced dataset, with only 4.87% responding positively to stroke. 
    <br>
</div>
''')

In [ ]:
HTML('''
<div class="alert alert-block alert-info">
    <b>Target Population:</b><br>
    The target population is mostly likely the group of people who are 
    concerned about their probability of getting a stroke.<br><br>
    
    <b>Sampling Population:</b><br>
    Our data's distribution describes our sampling population. 
    <br><br>
    
    <b>Potential Bias:</b><br>
    The data's collection method and answer's definition are unknown. 
    If the sampling population is not representative of the target 
    population, there might be significant biases. 
    <br>
</div>
''')

## 3.2. Distribution according to target label
Objective:<br>
Plot data distribution by target variable to understand if the populations 
show clear differences in their characteristics.

**Numerical Features** <br>
Cases with stroke

In [ ]:
stroke_data[stroke_data.stroke == 1][numerical_columns].describe()

Cases without stroke

In [ ]:
stroke_data[stroke_data.stroke == 0][numerical_columns].describe()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4), sharex=True)

violin_boxplot(data=stroke_data[stroke_data.stroke == 1], 
               columns=numerical_columns, title='stroke - True', ax=axes[1])
violin_boxplot(data=stroke_data[stroke_data.stroke == 0], 
               columns=numerical_columns, title='stroke - False', ax=axes[0])

plt.tight_layout()
plt.show()

In [ ]:

stacked_horizontal_feature_distribution(data=stroke_data[stroke_data.stroke == 1], 
                                        columns=categorical_columns, 
                                        title='Categorical Features '
                                              'Distribution\nstroke'
                                              ' - True') 
stacked_horizontal_feature_distribution(data=stroke_data[stroke_data.stroke == 0], 
               columns=categorical_columns, title='Categorical Features '
                                              'Distribution\nstroke - False')

In [ ]:
HTML('''
<div class="alert alert-block alert-info">
    <b>Distribution Insights:</b><br>
    Plots show visually significant difference between population with and 
    without stroke.<br><br>
    <i>numerical features...</i><br>
    1. With stroke consists of a higher age and higher bmi mean, with a 
    smaller standard deviation. Considering the sample size is smaller for 
    cases with stroke, it is very possible that these 2 features are highly 
    predictive of our target variable.<br>
    2. Average Glucose Level for cases with stroke has both the higher mean 
    and higher standard deviation. This is likely due to its clear 
    bimodal distribution. <br><br>

    
    <i>for categorical features...</i><br>
    3.  Smoking status, residence type, and gender seem similar among both 
    target labels. This could mean these features do not contain predictive 
    information for the target variable.<br>
    4. Work type, ever married, heart disease, and hypertension show 
    more different distribution.<br>
    5. Knowing that age seem to be an important factor, important to note 
    that some other features are also highly influenced by age. For example,
     it is more likely for the older age group to have a higher ratio to have 
    ever been married, or worked. <br>
</div>
''')

## 3.3. Missing values

Objective: <br>
All missing values come from the 'bmi' feature. Determine proper treatment.

Plot data distribution of cases with missing bmi values. Compare to the 
original data distributions, and check if there is a clear difference 
with missing bmi cases.

In [ ]:
violin_boxplot(data=stroke_data[stroke_data.bmi.isnull()], 
               columns=['age', 'avg_glucose_level'],
               title='Numerical Data Violin Box Plot\nbmi missing - True')

In [ ]:
stacked_horizontal_feature_distribution(data=stroke_data[stroke_data.bmi.isnull()], 
                                        columns=categorical_columns, 
                                        title='Numerical Data Violin Box'
                                              'Plot\nbmi missing - True')

The distribution for both numerical and categorical do not seem to represent
 a different subpopulation. <br>
Check if bmi is correlated with other features, this can help choose what 
values to impute.

In [ ]:
sns.lmplot(data=stroke_data, x='avg_glucose_level', y='bmi', hue='stroke')
sns.lmplot(data=stroke_data, x='age', y='bmi', hue='stroke')

In [ ]:
correlation_matrix = stroke_data[numerical_columns].corr()

plt.figure(figsize=(5, 4))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, 
            center=0)
plt.title('Pearson Correlation Matrix')
plt.show()

From the plots above: <br>
1. Correlation between numerical features are very weak to weak.
2. bmi values could be imputed by linear interpolation from other numerical 
features. Since correlation coefficient is low, using mean imputation or 
leaving it as null values is also good enough.

In [ ]:
HTML('''
<div class="alert alert-block alert-info">
    <b>Missing Values Insights:</b><br>
    After some analysis, it is understood that missing values do not belong 
    to any specific subpopulation. The subset of data which has missing 
    'bmi' values do not show a drastically different distribution in other 
    features. Therefore, imputing with the mean value, or even keeping it as
    null value during the preprocessing step will be enough.
</div>
''')

# 4. Data Formatting
Objective: Only decision tree based models will be used in this project, 
so the following formatting is only to be able to draw a tree visually, 
providing with better interpretability in the end.

Remove all empty values in values.

In [ ]:
stroke_data['smoking_status'] =(
    stroke_data['smoking_status'].replace(' ', '_', regex=True))
stroke_data['smoking_status'].unique()

Set empty values to 0 

In [ ]:
stroke_data['bmi'] = stroke_data['bmi'].fillna(0)

# 5. Preprocessing
Objective:<br>
Prepare preprocesses in order to apply learning models. <br>
1. Data splitting
2. Data transformation and encoding

## 5.1. Data Splitting

In [ ]:
X, y = (stroke_data.drop(columns=[target_name]), 
                stroke_data[target_name])

Split data into train, validation, and test sets. With a ratio of 20% being 
the test set.

In [ ]:
X_train_val, X_test, y_train_val, y_test =\
    train_test_split(X, y, test_size=0.2, shuffle=True, random_state=0)

To evaluate variability of our estimation of the performance, cross 
validation will be used. More specifically, Stratified KFold, for the train 
and validation sets, as this is a better option for imbalanced and 
small datasets. 

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

## 5.2. Data Transformation

For models based on decision trees, transformations like scaling and 
normalizing will not be needed, only encoding categorical columns will be 
necessary. But we would want to compare results with a basic logistic 
regression, so we will apply StandardScaler().

**Define columns**

In [ ]:
multicat_columns = ['gender', 'work_type', 'smoking_status']
binary_columns = ['hypertension', 'heart_disease', 'ever_married', 
                 'Residence_type']

**Define preprocessors for each type of data**<br>
LabelEncoder does not work within a pipeline, write our own function.

In [ ]:
multicat_preprocessor = OneHotEncoder(handle_unknown='ignore')
binary_preprocessor = FunctionTransformer(binary_label_encoding)
numeric_preprocessor = StandardScaler()

**Bundle preprocessing for all data**

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', multicat_preprocessor, multicat_columns),
        ('binary', binary_preprocessor, binary_columns),
        ('numeric', numeric_preprocessor, numerical_columns)
    ]
)

# 6. Models
Objective: <br>
1. Apply and hyperparameter tuning machine learning models
2. Assess and select the best model

**Models**
1. Random forest classifier
2. XGBoost gradient boosted trees
3. LightGBM histogram gradient boosting
4. Logistic regression

In [ ]:
random_forest = RandomForestClassifier(random_state=0)
clf_xbg = xgb.XGBClassifier(objective='binary:logistic', missing=0, 
                            random_state=0)
clf_lgb = lgb.LGBMClassifier(objective='binary', random_state=0)
log_reg = LogisticRegression(random_state=0)

In [ ]:
stroke_data.isnull().any().any()

**Pipeline**

In [ ]:
pipeline_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', random_forest)
])

pipeline_xgb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', clf_xbg)
])

pipeline_lgb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', clf_lgb)
])

pipeline_lr = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', log_reg)
])

## 6.1. Basic model
Objective: Fit and evaluate the basic model with default parameters to gain 
intuition on what improvements need to be made.

In [ ]:
cv_results_rf = cross_validate(
    pipeline_rf, X_train_val, y_train_val, cv=skf,
    scoring=['roc_auc', 'accuracy', 'precision', 'recall', 'f1'],
    return_train_score=True, n_jobs=2
)

cv_results_xgb = cross_validate(
    pipeline_xgb, X_train_val, y_train_val, cv=skf,
    scoring=['roc_auc', 'accuracy', 'precision', 'recall', 'f1'], 
    return_train_score=True, n_jobs=2
)

cv_results_lgb = cross_validate(
    pipeline_lgb, X_train_val, y_train_val, cv=skf,
    scoring=['roc_auc', 'accuracy', 'precision', 'recall', 'f1'], 
    return_train_score=True, n_jobs=2
)

cv_results_lr = cross_validate(
    pipeline_lr, X_train_val, y_train_val, cv=skf,
    scoring=['roc_auc', 'accuracy', 'precision', 'recall', 'f1'], 
    return_train_score=True, n_jobs=2
)

print(f"cv_results_rf{cv_results_rf}")
print(f"cv_results_xgb{cv_results_xgb}")
print(f"cv_results_lgb{cv_results_lgb}")
print(f"cv_results_lr{cv_results_lr}")

In [ ]:
df_rf = pd.DataFrame(cv_results_rf).mean().to_frame('Random Forest')
df_xgb = pd.DataFrame(cv_results_xgb).mean().to_frame('XGBoost')
df_lgb = pd.DataFrame(cv_results_lgb).mean().to_frame('LightGBM')
df_lr = pd.DataFrame(cv_results_lr).mean().to_frame('Logistic Regression')

results_df = pd.concat([df_rf, df_xgb, df_lgb, df_lr], axis=1)
results_df

In [ ]:
HTML('''
<div class="alert alert-block alert-info">
 <b>Fit and Score Time:</b><br>
    XGBoost is also the fastest in terms of score time, followed by 
LightGBM, and Random Forest is slightly slower. XGBoost and LightGBM are 
designed with optimizations that make predictions very fast, which explains 
why their scoring times are lower than Random Forest. LightGBM is also 
efficient but may benefit from more tuning, as its fit and score time are 
unexpectedly higher than XGBoost. <br><br>
<b>Test Scores:</b><br>
<i>For a problem like ours where main goal is to predict 
whether a person is likely or not to have stroke, it is preferable to have 
more FALSE POSITIVES than FALSE NEGATIVES.</i><br>
1. Accuracy: In our dataset due to how imbalanced it is, accuracy is not a good
indicator to rely on. <br>
2. Precision: This metric represents the accuracy of the positive 
predictions. This value is consistently low on all models, being 0.23 from 
XGBoost the highest. Considering the nature of the problem, this is not a 
huge issue. <br>
3. Recall: This is one of the most important metric in our case, as it 
captures how often the model correctly identifies a case with stroke from all 
cases with stroke. Currently this value is extremely low on all models.<br>
4. Area under the curve (AUC ROC): This metric measures how well the model 
distinguishes between positive and negative cases. LightGBM has the best 
results of all 3.<br>
5. F1 Score: This metric balances how well both recall and precision scored.
 Even though precision is not the focus of our problem, it is still ideal to
  have a model which performs better on both. And it is currently scoring 
  low on all models.
</div>
''')

## 6.2. Baseline model
Objective: Our dataset is very imbalanced, more than 95% of data belongs to 
no stroke (0). If the model simply predicted every case with no stroke, it 
would still get a very high score. Therefor, we will compare obtained scores
 to this baseline, to understand how much better actually are the models.

In [ ]:
baseline_model = DummyClassifier(strategy='constant', constant=0)

baseline_results = cross_validate(
    baseline_model, X_train_val, y_train_val, cv=skf, 
    scoring={
        'roc_auc': 'roc_auc',
        'accuracy': 'accuracy',
        'precision': make_scorer(precision_score, zero_division=0),
        'recall': 'recall',
        'f1': make_scorer(f1_score, zero_division=0)
    }, 
    return_train_score=True, n_jobs=2
)
baseline_df = pd.DataFrame(baseline_results).mean().to_frame('Baseline')
baseline_df

In [ ]:
HTML('''
<div class="alert alert-block alert-info">
 <b>Baseline Model:</b><br>
 As expected, if the model predicted all values to be 0, no stroke, the 
 accuracy score would be very high, 0.95 in this case is actually higher 
 than XGBoost and LightGBM. <br>As the 
 DummyClassifier does not distinguish between positive and negative cases, 
 it is also predictable that AUC is 0.5, as good as random guesses would do.
 </div>
''')

## 6.3. Hyperparameter tuning

**First tuning**

The first hyperparameter tuning resulted in the following **best 
parameters** and **best recall scores**: <br><br>
*It is clear that the models are not good at identifying the actual 
positive cases, which is the goal of the project.*

1. param_grid_rf = { <br>
    'classifier__n_estimators': [100, 200], <br>
    'classifier__max_depth': [5, 10, None], <br>
    'classifier__min_samples_split': [2, 10],<br>
    'classifier__min_samples_leaf': [1, 5] <br>
}<br>
Random Forest Best Parameters: {'classifier__max_depth': 10, 
'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 
'classifier__n_estimators': 100} <br><br>

2. param_grid_xgb = {<br>
    'classifier__n_estimators': [100, 200], <br>
    'classifier__max_depth': [3, 6, 10], <br>
    'classifier__learning_rate': [0.01, 0.1], <br>
    'classifier__subsample': [0.8, 1.0]<br>
}<br>
XGBoost Best estimator: {'classifier__learning_rate': 0.1, 
'classifier__max_depth': 6, 'classifier__n_estimators': 100, 
'classifier__subsample': 0.8}<br><br>

3. param_grid_lgb = {<br>
    'classifier__n_estimators': [100, 200],<br>
    'classifier__max_depth': [5, 10],<br>
    'classifier__learning_rate': [0.01, 0.1],<br>
    'classifier__num_leaves': [31, 50]<br>
}<br>
LightGBM Best estimator: {'classifier__learning_rate': 0.1, 
'classifier__max_depth': 5, 'classifier__n_estimators': 200, 
'classifier__num_leaves': 31}<br>


Random Forest Best recall score: 0.020512820512820513<br>
XGBoost Best recall score: 0.0923076923076923<br>
LightGBM Best recall score: 0.08205128205128204<br>

**Second tuning**<br>

1. Random Forest Best estimator: {'classifier__max_depth': 12, 
'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 
'classifier__n_estimators': 50} <br>
2. XGBoost Best estimator: {'classifier__learning_rate': 0.12, 
'classifier__max_depth': 4, 'classifier__n_estimators': 120, 
'classifier__subsample': 0.8} <br>
3. LightGBM Best estimator: {'classifier__learning_rate': 0.15, 
'classifier__max_depth': 5, 'classifier__n_estimators': 250, 
'classifier__num_leaves': 20} <br>

Random Forest Best recall score: 0.03076923076923077 <br>
XGBoost Best recall score: 0.10256410256410256<br>
LightGBM Best recall score: 0.10256410256410256<br>

**Third tuning**<br>

1. Random Forest Best estimator: {'classifier__max_depth': 15, 
'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 3, 
'classifier__n_estimators': 30} <br>
2. XGBoost Best estimator: {'classifier__learning_rate': 0.12, 
'classifier__max_depth': 4, 'classifier__n_estimators': 150, 
'classifier__subsample': 0.7} <br>
3. LightGBM Best estimator: {'classifier__learning_rate': 0.2, 
'classifier__max_depth': 5, 'classifier__n_estimators': 220, 
'classifier__num_leaves': 15} <br>

Random Forest Best recall score: 0.035897435897435895<br>
XGBoost Best recall score: 0.10769230769230768<br>
LightGBM Best recall score: 0.11794871794871793<br>

**Forth tuning**

In [ ]:
param_grid_rf = { 
    'classifier__n_estimators': [20, 30, 50],
    'classifier__max_depth': [12, 15],
    'classifier__min_samples_split': [2, 3],
    'classifier__min_samples_leaf': [1, 2]
}


param_grid_xgb = {
    'classifier__n_estimators': [150, 200],
    'classifier__max_depth': [2, 4, 5],
    'classifier__learning_rate': [0.11, 0.12, 0.13],
    'classifier__subsample': [0.6, 0.7, 0.8]
}

param_grid_lgb = {
    'classifier__n_estimators': [200, 220, 250],
    'classifier__max_depth': [4, 5, 6],
    'classifier__learning_rate': [0.15, 0.2, 0.3],
    'classifier__num_leaves': [5, 10, 15]
}

In [ ]:
scoring_metrics = {
    'roc_auc': 'roc_auc',
    'precision': make_scorer(precision_score, zero_division=0),
    'recall': 'recall',
    'f1': make_scorer(f1_score, zero_division=0)
}

grid_rf = GridSearchCV(pipeline_rf, param_grid_rf, cv=skf, 
                       scoring=scoring_metrics, refit='recall', verbose=10)
grid_xgb = GridSearchCV(pipeline_xgb, param_grid_xgb, cv=skf, 
                        scoring=scoring_metrics, refit='recall', verbose=10)
grid_lgb = GridSearchCV(pipeline_lgb, param_grid_lgb, cv=skf, 
                        scoring=scoring_metrics, refit='recall', verbose=10)

grid_rf.fit(X_train_val, y_train_val)
grid_xgb.fit(X_train_val, y_train_val)
grid_lgb.fit(X_train_val, y_train_val)

print("Random Forest Best estimator:", grid_rf.best_params_)
print("XGBoost Best estimator:", grid_xgb.best_params_)
print("LightGBM Best estimator:", grid_lgb.best_params_)

Print out recall score for the best parameters found after 4 iterations of 
Hyperparameter Tuning.

In [ ]:
print("Random Forest Best recall score:", grid_rf.best_score_)
print("XGBoost Best recall score:", grid_xgb.best_score_)
print("LightGBM Best recall score:", grid_lgb.best_score_)

Save the newly fitted model's parameters

In [ ]:
best_rf_2 = grid_rf.best_estimator_
best_xgb_2 = grid_xgb.best_estimator_
best_lgb_2 = grid_lgb.best_estimator_

In [ ]:
HTML('''
<div class="alert alert-block alert-info">
 <b>Hyperparameter Tuning:</b><br>
 After 4 iterations of tuning hyperparameters, it has resulted in better 
 performance of the model with the training and validation set.<br><br>
 The best parameters the models have at this stage: <br>
 1. Random Forest Best estimator: {'classifier__max_depth': 15, 
 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 3, 
 'classifier__n_estimators': 30}<br>
2. XGBoost Best estimator: {'classifier__learning_rate': 0.12, 
'classifier__max_depth': 4, 'classifier__n_estimators': 200, 
'classifier__subsample': 0.8}<br>
3. LightGBM Best estimator: {'classifier__learning_rate': 0.3, 
'classifier__max_depth': 6, 'classifier__n_estimators': 220, 
'classifier__num_leaves': 10}<br> <br>

With the following recall score on validation set:<br>
1. Random Forest Best recall score: 0.035897435897435895<br>
2. XGBoost Best recall score: 0.12307692307692308<br>
3. LightGBM Best recall score: 0.1282051282051282<br><br>

In general:<br>
XGBoost and LightGBM have very similar recall scores with the validation 
set, 12.3% and 12.8% respectively. Random Forest had the worst scoring, 
almost with only 3.60%. Considering that recall is an important metric in 
our case, RandomForest would not be considered for further comparison.<br><br>
 </div>
''')

## 6.4. Select the best model


In [ ]:
y_pred_xgb = best_xgb_2.predict(X_test)
y_pred_lgb = best_lgb_2.predict(X_test)

In [ ]:
print("Random Forest - Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Random Forest - ROC AUC:", roc_auc_score(y_test, y_pred_rf))
print("Random Forest - Precision:", precision_score(y_test, y_pred_rf))
print("Random Forest - Recall:", recall_score(y_test, y_pred_rf))
print("Random Forest - F1 Score:", f1_score(y_test, y_pred_rf))

In [ ]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")
    print(f"Precision: {precision_score(y_test, y_pred):.3f}")
    print(f"Recall: {recall_score(y_test, y_pred):.3f}")
    print(f"F1-Score: {f1_score(y_test, y_pred):.3f}")

print("Random Forest Performance:")
evaluate_model(grid_rf, X_test, y_test)

print("\nXGBoost Performance:")
evaluate_model(grid_xgb, X_test, y_test)

print("\nLightGBM Performance:")
evaluate_model(grid_lgb, X_test, y_test)

# 12. Print best parameters for each model
print("Best Parameters for Random Forest:", grid_rf.best_params_)
print("Best Parameters for XGBoost:", grid_xgb.best_params_)
print("Best Parameters for LightGBM:", grid_lgb.best_params_)

## 6.5. Feature Importance

In [ ]:
import matplotlib.pyplot as plt
importances = best_rf_model.named_steps['classifier'].feature_importances_
plt.barh(range(len(importances)), importances)
plt.show()

## Models
1. XGBoost
2. LightBGM
3. Single decision tree - sklearn
4. Bagging - sklearn
5. Random Forest - sklearn

Draw Tree in the end for the best performance model

# 7. Deploy the machine learning model

# Improvement

1. Our data is imbalanced and can show certain sampling biases. This will 
increase the model's incorrect predictions. A more representative and 
careful data collection with proper definitions will help.
2. More related data about symptoms/discomfort/previous medical history can 
help develop a model that is better adjusted for the use case.
3. T-test and Chi-square test to thoroughly test differences between stroke/no 
stroke and bmi subpopulation.
4. Feature engineering - split bimodal avg_glucose_level into 2 features
5. To address imbalanced data try resampling techniques, like oversampling and 
under sampling; try adjusting class weights; try adjusting decision threshold
6. 